In [60]:
orthophotos[4][-26:]

'35-2006-0305-6780-LA93.tif'

In [1]:
import pandas as pd
from PIL import Image
from IPython.display import display
import glob,os
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
from skimage.transform import resize, downscale_local_mean
import scipy
import cv2
from tqdm import tqdm
from time import time

In [2]:
ortho_path = {
    '35': {
        '2020': "donnees/BDORTHO/ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D035_2020-01-01/ORTHOHR/1_DONNEES_LIVRAISON_2021-06-00121/OHR_RVB_0M20_JP2-E080_LAMB93_D35-2020/",
        '2017': "donnees/BDORTHO/BDORTHO_2-0_RVB-0M50_JP2-E080_LAMB93_D035_2017-01-01/BDORTHO/1_DONNEES_LIVRAISON_2019-01-00118/BDO_RVB_0M50_JP2-E080_LAMB93_D35-2017/",
        '2012': "donnees/BDORTHO/BDORTHO_2-0_RVB-0M50_JP2-E080_LAMB93_D035_2012-01-01/BDORTHO/1_DONNEES_LIVRAISON_2015-02-00365/BDO_RVB_0M50_JP2-E080_LAMB93_D35-2012/"
    },
    '49': {
        '2020': "donnees/BDORTHO/ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D049_2020-01-01/ORTHOHR/1_DONNEES_LIVRAISON_2020-12-00085/OHR_RVB_0M20_JP2-E080_LAMB93_D49-2020/",
    }
}

irc_path = {
    '35': {
        '2020': "donnees/BDORTHO_IRC/ORTHOHR_1-0_IRC-0M20_JP2-E080_LAMB93_D035_2020-01-01/ORTHOHR/1_DONNEES_LIVRAISON_2021-06-00110/OHR_IRC_0M20_JP2-E080_LAB93_D35-2020/",
    },
    '49': {
        '2020': "donnees/BDORTHO_IRC/ORTHOHR_1-0_IRC-0M20_JP2-E080_LAMB93_D049_2020-01-01/ORTHOHR/1_DONNEES_LIVRAISON_2020-12-00132/OHR_IRC_0M20_JP2-E080_LAMB93_D49-2020/",
    }
}

config = {
    "mnhc_path":"donnees/MNHC/",
    "ortho_path": ortho_path, # "données/BDORTHO/ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D035_2020-01-01/ORTHOHR/1_DONNEES_LIVRAISON_2021-06-00121/OHR_RVB_0M20_JP2-E080_LAMB93_D35-2020/",# "données/orthophotos/2006_OK/rvb/",
    "irc_path": irc_path, # "données/BDORTHO_IRC/ORTHOHR_1-0_IRC-0M20_JP2-E080_LAMB93_D035_2020-01-01/ORTHOHR_1-0_IRC-0M20_JP2-E080_LAMB93_D035_2020-01-01/ORTHOHR/1_DONNEES_LIVRAISON_2021-06-00110/OHR_IRC_0M20_JP2-E080_LAMB93_D35-2020/",# "données/orthophotos/2006_OK/rvb/",
    'rgb_vignettes_path':'vignettes/rgb/',
    'mask_vignettes_path':'vignettes/mask/',
    'rgb_coordinates_pos':-27,
    'rgb_name_pos':-36,
    'irc_pos': -9,
    'img_size':256
}

In [17]:
import json
with open("config.json", "w") as outfile:
    json.dump(config, outfile)

In [18]:
with open('config.json') as f_in:
    config_2 = json.load(f_in)

In [3]:
path_to_orthophoto_rgb = config['ortho_path']['35']['2020'] + '35-2020-0305-6780-LA93-0M20-E080.jp2'

In [4]:
def std_convoluted(im, N):
    """
    Calculate the standard deviation using a convolution operation.

    Parameters:
    - im (numpy.ndarray): Input image as a NumPy array.
    - N (int): Radius of the convolution kernel.

    Returns:
    - numpy.ndarray: Standard deviation image.
    """
    # Calculate squared image and initialize kernel
    im2 = im**2
    ones = np.ones(im.shape)
    kernel = np.ones((2*N+1, 2*N+1))

    # Convolve the image and squared image with the kernel
    s = scipy.signal.convolve2d(im, kernel, mode="same")
    s2 = scipy.signal.convolve2d(im2, kernel, mode="same")
    ns = scipy.signal.convolve2d(ones, kernel, mode="same")

    # Calculate the standard deviation image
    return np.sqrt((s2 - s**2 / ns) / ns)

In [5]:
path_to_orthophoto_rgb[config['rgb_coordinates_pos']:config['rgb_coordinates_pos']+8]

'305-6780'

In [6]:
os.path.basename(path_to_orthophoto_rgb[config['rgb_name_pos']:])[:2]

'35'

In [7]:
os.path.basename(path_to_orthophoto_rgb)[3:7]

'2020'

In [13]:
def my_func(path_to_orthophoto_rgb):
    
    Image.MAX_IMAGE_PIXELS = None
    
    rgb_name = os.path.basename(path_to_orthophoto_rgb)
    dept = rgb_name[:2]
    year = rgb_name[3:7]
    path_to_orthophoto_irc = config["irc_path"][dept][year] + rgb_name[:config['irc_pos']] + '-IRC' + rgb_name[config['irc_pos']:]# config["irc_path"] + path_to_orthophoto_rgb[config['rgb_name_pos']:] # A Changer plus tard
    # mnhc_tiff = config['mnhc_path'] + "Diff_MNS_CORREL_1-0_LAMB93_20FD3525_" + str(path_to_orthophoto_rgb[config['rgb_coordinates_pos']:config['rgb_coordinates_pos']+8]).replace('-', '_') + ".tif"
    
    # position
    rgb_x = path_to_orthophoto_rgb[config['rgb_coordinates_pos']: config['rgb_coordinates_pos']+3]
    rgb_y = path_to_orthophoto_rgb[config['rgb_coordinates_pos']+4: config['rgb_coordinates_pos']+8]
    mnhc_dir_dept = config['mnhc_path']+dept+'_'+year+'/'
    
    # images IRC et RGB
    # ortho_irc_file = config["irc_path"] + '/' + dept + '-2020-0'+rgb_x+'-'+rgb_y+'-LA93-0M20-IRC-E080.jp2'
    # ouverture
    ortho_rgb = cv2.resize(np.asarray(Image.open(path_to_orthophoto_rgb)), (10000, 10000), interpolation=cv2.INTER_AREA)
    ortho_irc = cv2.resize(np.asarray(Image.open(path_to_orthophoto_irc)), (10000, 10000), interpolation=cv2.INTER_AREA) 
    ndvi = np.divide(ortho_irc[:,:,0]-ortho_irc[:,:,1],ortho_irc[:,:,0]+ortho_irc[:,:,1], where=(ortho_irc[:,:,0]+ortho_irc[:,:,1])!=0 )
    ortho_irc = None 
    
    if not os.path.exists(config['rgb_vignettes_path'] + f'rgb_{rgb_x}_{rgb_y}/'):
        os.makedirs(config['rgb_vignettes_path'] + f'rgb_{rgb_x}_{rgb_y}/')
    if not os.path.exists(config['mask_vignettes_path'] + f'mask_{rgb_x}_{rgb_y}/'):
        os.makedirs(config['mask_vignettes_path'] + f'mask_{rgb_x}_{rgb_y}/')

    # i = 0
    vignette_data = []
    for mnhc_x in tqdm(range(0,5)):
        for mnhc_y in range(0,5):
            mnhc_file = mnhc_dir_dept+'Diff_MNS_CORREL_1-0_LAMB93_20FD'+dept+'25_'+str(int(int(rgb_x)+mnhc_x))+'_'+str(int(int(rgb_y)-mnhc_y))+'.tif'
            
            if os.path.exists(mnhc_file):
                # print(mnhc_file)
                mnhc = np.asarray(Image.open(mnhc_file))
                w, h = mnhc.shape
                mask = np.logical_and(np.logical_and(mnhc > 3., 
                                                     np.logical_or(mnhc > 5., 
                                                                   std_convoluted(mnhc,N=10) > .5)
                                                    ),
                                      ndvi[mnhc_x*w:(mnhc_x+1)*w, mnhc_y*h:(mnhc_y+1)*h] > 0) \
                .astype('uint8')

                
                for x in range(0, w-config['img_size']+1, config['img_size']):
                    for y in range(0, h-config['img_size']+1, config['img_size']):
                        # print(x, y)
                        crop_rgb = ortho_rgb[mnhc_y*h+y: mnhc_y*h+y+config['img_size'], mnhc_x*w+x: mnhc_x*w+x+config['img_size'], :]
                        crop_mask = mask[y:y+config['img_size'], x:x+config['img_size']]

                        # filtre rgb
                        # enregistrement
                        pos = dept+'_'+str(int((int(rgb_x)+mnhc_x)*1000+x/2))+'_'+str(int((int(rgb_y)-mnhc_y)*1000+y/2))
                        
                        cv2.imwrite(config['rgb_vignettes_path'] + f'rgb_{rgb_x}_{rgb_y}/' + 'rgb_' + str(pos) + '.jpg', crop_rgb)
                        cv2.imwrite(config['mask_vignettes_path'] + f'mask_{rgb_x}_{rgb_y}/' + 'mask_'+ str(pos) + '.png', crop_mask)

                        # Calculate characteristics of the RGB crop and add to vignette_data
                        mask_2 = np.stack((crop_mask,)*3, axis=-1)
                        non_mask = np.logical_not(mask_2)
                        res = [pos, crop_mask.sum()] + \
                        np.mean(crop_rgb, axis=(0, 1), where=mask_2.astype(bool)).tolist() + \
                        np.mean(crop_rgb, axis=(0, 1), where=non_mask.astype(bool)).tolist() + \
                        np.std(crop_rgb, axis=(0, 1), where=mask_2.astype(bool)).tolist() + \
                        np.std(crop_rgb, axis=(0, 1), where=non_mask.astype(bool)).tolist()
                        vignette_data.append(res)
                
                # Si besoin de plus d'images, commenter le break
                break
    
    # Convert the vignette data to a Pandas DataFrame and return
    return pd.DataFrame(vignette_data, columns=('file', 'wood_size', 'w_mean_R', 'w_mean_G', 'w_mean_B', 'nw_mean_R', 'nw_mean_G', 'nw_mean_B', 'w_std_R', 'w_std_G', 'w_std_B', 'nw_std_R', 'nw_std_G', 'nw_std_B'))
                

In [9]:
rgb_name = os.path.basename(path_to_orthophoto_rgb)
dept = rgb_name[:2]
year = rgb_name[3:7]
path_to_orthophoto_irc = config["irc_path"][dept][year] + rgb_name[:config['irc_pos']] + '-IRC' + rgb_name[config['irc_pos']:]

In [10]:
path_to_orthophoto_irc

'donnees/BDORTHO_IRC/ORTHOHR_1-0_IRC-0M20_JP2-E080_LAMB93_D035_2020-01-01/ORTHOHR/1_DONNEES_LIVRAISON_2021-06-00110/OHR_IRC_0M20_JP2-E080_LAB93_D35-2020/35-2020-0305-6780-LA93-0M20-IRC-E080.jp2'

In [11]:
path_to_orthophoto_rgb = 'donnees/BDORTHO/ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D035_2020-01-01/ORTHOHR/1_DONNEES_LIVRAISON_2021-06-00121/OHR_RVB_0M20_JP2-E080_LAMB93_D35-2020/35-2020-0315-6785-LA93-0M20-E080.jp2'


In [14]:
start = time()
df = my_func(path_to_orthophoto_rgb)
print(f'vignette creation done in {round(time()-start, 2)} sec')

  0%|                                    | 0/5 [00:00<?, ?it/s]C:\Users\adelabrosse\AppData\Local\Temp\ipykernel_14160\4112585707.py:23: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt((s2 - s**2 / ns) / ns)
C:\Users\adelabrosse\.conda\envs\inrae_seg_env\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\adelabrosse\.conda\envs\inrae_seg_env\Lib\site-packages\numpy\core\_methods.py:184: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
C:\Users\adelabrosse\.conda\envs\inrae_seg_env\Lib\site-packages\numpy\core\_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\adelabrosse\.conda\envs\inrae_seg_env\Lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
C:\Users\adelabro

vignette creation done in 99.48 sec


In [15]:
df

,file,wood_size,w_mean_R,w_mean_G,w_mean_B,nw_mean_R,nw_mean_G,nw_mean_B,w_std_R,w_std_G,w_std_B,nw_std_R,nw_std_G,nw_std_B
0,35_315000_6785000,28418,62.454536,74.516117,61.784151,96.457218,100.838569,82.293146,22.259598,20.490851,14.232360,34.414154,24.696099,21.754726
1,35_315000_6785128,56284,58.530097,71.008795,60.042072,65.166450,79.047341,62.710981,14.477308,14.373316,10.929631,17.805755,16.476592,8.682647
2,35_315000_6785256,53197,54.981352,67.623325,57.975318,80.077154,95.023746,70.671286,15.330876,14.844511,10.309024,24.818022,16.590431,11.900248
3,35_315000_6785384,3306,63.640048,77.572595,63.716273,71.548739,86.755439,68.333826,15.756308,15.431827,10.135857,17.137923,11.890340,9.841731
4,35_315000_6785512,4786,70.954450,79.942123,67.543460,72.280444,86.703605,70.042486,26.773477,22.605508,14.803557,22.078805,17.454904,17.383212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,35_319768_6785256,65355,59.548634,71.635973,59.422722,73.408840,81.944751,67.972376,23.720431,24.311894,16.988984,33.602301,27.223390,24.034835
241,35_319768_6785384,65389,54.157182,65.779275,56.057349,75.802721,78.306122,66.619048,21.904906,22.374061,15.370922,39.698976,28.058154,23.664990
242,35_319768_6785512,65385,55.616426,65.503617,57.106691,54.913907,64.761589,56.423841,21.300550,21.502283,15.460915,20.161579,20.894154,14.736735
243,35_319768_6785640,65380,55.216060,65.139018,56.848792,60.141026,68.352564,61.576923,22.157005,22.225672,16.028925,28.380078,24.885585,21.855011


In [16]:
import sweetviz as sv

my_report = sv.analyze(df)
my_report.show_html()

C:\Users\adelabrosse\.conda\envs\inrae_seg_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Feature: file                                | | [  7%]   00:00C:\Users\adelabrosse\.conda\envs\inrae_seg_env\Lib\site-packages\sweetviz\from_profiling_pandas.py:65: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(keys):
Done! Use 'show' commands to display/save.   |█| [100%]   00:00


Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [39]:
mask = np.stack((crop_mask, )*3, axis=-1)
non_mask = np.logical_not(mask)
res = [pos, crop_mask.sum()] + \  # ??? Add computed values here
       np.mean(crop_rgb, axis=(0, 1), where=mask).tolist() + \
       np.mean(crop_rgb, axis=(0, 1), where=non_mask).tolist() + \
       np.std(crop_rgb, axis=(0, 1), where=mask).tolist() + \
       np.std(crop_rgb, axis=(0, 1), where=non_mask).tolist()



img opening done in 63.39 sec
